In [6]:
import os
import pandas as pd
import re
import datetime

def ts2dt(x):
    return datetime.datetime.utcfromtimestamp(x)

def filter_df(data_path):
    pattern = r'.*/script_rpc_blocks_(\d+)_(\d+)\.csv'
    match = re.match(pattern, data_path)
    if match:
        start_block = int(match.group(1))
        end_block = int(match.group(2))
        new_df_name = '{}/filtered_rpc_blocks_{}_{}.csv'.format(data_path.rsplit('/', 1)[0], {}, {})
        df = pd.read_csv(data_path).sort_values(by='time')
        df['rtime'] = df['time'].map(ts2dt)
        len_old = len(df)
        print(f"Загружен датафрейм {data_path}")
        print(f"Длина датафрейма {len_old}")
        check = pd.read_csv('./check_btc.csv', index_col=0)
        check['ltime'] = pd.to_datetime(check['ltime'])
        check = check.sort_values(by='ltime')
        df = pd.merge_asof(df, check, left_on='rtime', right_on='ltime', direction='nearest')
        df = df[df['ltime'] - df['rtime'] < datetime.timedelta(minutes=5)]
        len_new = len(df)
        print(f"Новая длина {len_new}. Отфильровано {len_old - len_new}")
        df.to_csv(new_df_name.format(start_block, end_block), index=False)


In [7]:
data_path = "part1/script_rpc_blocks_834577_834660.csv"

In [8]:
filter_df(data_path)

Загружен датафрейм part1/script_rpc_blocks_834577_834660.csv
Длина датафрейма 175078
Новая длина 129356. Отфильровано 45722


In [113]:
df = pd.read_csv(data_path)

In [114]:
df = df.head(10000)

In [115]:
len(df)

10000

In [116]:
check = pd.read_csv('./check_btc.csv', index_col=0)

In [117]:
check['ltime'] = pd.to_datetime(check['ltime'])

In [118]:
check = check.sort_values(by='ltime')

In [119]:
len(check)

4300

In [120]:
check

,ltime,btcsum
116250,2024-02-29 21:06:53,0.000883
83376,2024-02-29 21:07:09,0.001767
165801,2024-02-29 21:13:19,0.000353
116267,2024-02-29 21:19:29,0.000883
132860,2024-02-29 21:41:50,0.000447
...,...,...
66759,2024-03-31 19:24:05,0.000153
66736,2024-03-31 19:25:06,0.000611
66744,2024-03-31 19:31:22,0.000458
66738,2024-03-31 19:31:37,0.000611


In [121]:
df['rtime'] = df['time'].map(ts2dt)
df = df.sort_values(by='rtime')

In [122]:
ind = 3480
check['ltime'][ind:ind + 20]

33035    2024-03-14 10:34:06
49931    2024-03-14 11:01:54
610427   2024-03-14 11:21:29
593910   2024-03-14 11:36:48
33233    2024-03-14 12:32:39
29       2024-03-14 12:40:59
66483    2024-03-14 12:43:49
49885    2024-03-14 12:44:40
66386    2024-03-14 12:45:04
33373    2024-03-14 12:45:18
49874    2024-03-14 12:45:21
23       2024-03-14 12:45:31
49801    2024-03-14 12:46:00
41       2024-03-14 12:46:06
46       2024-03-14 12:50:37
44       2024-03-14 12:51:18
20       2024-03-14 12:51:28
40       2024-03-14 12:51:33
1        2024-03-14 12:51:44
21       2024-03-14 12:52:45
Name: ltime, dtype: datetime64[ns]

In [123]:
df['rtime'][:10]

0      2024-03-14 12:14:26
1732   2024-03-14 12:14:26
1733   2024-03-14 12:14:26
1734   2024-03-14 12:14:26
1735   2024-03-14 12:14:26
1736   2024-03-14 12:14:26
1737   2024-03-14 12:14:26
1738   2024-03-14 12:14:26
1731   2024-03-14 12:14:26
1739   2024-03-14 12:14:26
Name: rtime, dtype: datetime64[ns]

In [124]:
import datetime
def ts2dt(x):
    return datetime.datetime.utcfromtimestamp(x)

In [127]:
df = pd.merge_asof(df, check, left_on='rtime', right_on='ltime', direction='nearest')

In [128]:
# df['rtime'] = pd.merge_asof(df, check, left_on='ltime', right_on='ltime', direction='nearest')

In [129]:
df.head()

,in_active_chain,txid,hash,version,size,vsize,weight,locktime,vin,vout,hex,blockhash,confirmations,time,blocktime,blockNumber,rtime,ltime,btcsum
0,True,cc3d2ae9ed7f719dccc5475638ace7cfce03af7376f573...,698c5a5b76a9a58599060741559bc898ab5b585dee3791...,1,396,369,1476,0,[{'coinbase': '0365bc0c1d506f77657265642062792...,"[{'value': 5.46e-06, 'n': 0, 'scriptPubKey': {...",0100000000010100000000000000000000000000000000...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
1,True,08e994436242dd2206abae88518f169d62d1ed3fe6c17f...,bce4defb2a436d4be7197a952c10b4e9ce2edc039c53f4...,1,223,141,562,0,[{'txid': 'c8e8671a2abf065f365ad7de9eca8b88b94...,"[{'value': 0.00042065, 'n': 0, 'scriptPubKey':...",010000000001011513483e594fdf4985f85bab9da840b9...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
2,True,d4215e88e7ea22aa6167469973436999bd86d9e1f1fad7...,cae31fab4c872abb4a08882aedd09d4184939eb8f3f036...,1,520,277,1105,0,[{'txid': '5504708a3e0484f27790808ef4054ab115e...,"[{'value': 0.01249804, 'n': 0, 'scriptPubKey':...",01000000000103e5dbb73a99683b8b02cac55c9508e415...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
3,True,38e14aa2f30b0114d5b010c6457b18531d035fa450b0c4...,020be95be7695aeaa03fd405996b47412ee796c3a39fa5...,1,193,111,442,0,[{'txid': '08512c45c4a4dc6fc3267394fc085593764...,"[{'value': 0.02707927, 'n': 0, 'scriptPubKey':...",01000000000101350e2752c2d682c74fe977d7dab34076...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601
4,True,5e6e893616b08a16d690d615ba7048c404855828f7c789...,28509bbcbce90dcddaa296426f64de151ec55ebbf7e1e4...,1,404,213,851,0,[{'txid': '2716ba171d41efdd949218de1dcf87f7b18...,"[{'value': 0.00247116, 'n': 0, 'scriptPubKey':...",010000000001013d2b4bb50189eefe290dcb85e74180b1...,00000000000000000001765727b18bf7604ce8e017504f...,10850,1710418466,1710418466,834661,2024-03-14 12:14:26,2024-03-14 12:32:39,0.000601


In [130]:
check.head(40)

,ltime,btcsum
116250,2024-02-29 21:06:53,0.000883
83376,2024-02-29 21:07:09,0.001767
165801,2024-02-29 21:13:19,0.000353
116267,2024-02-29 21:19:29,0.000883
132860,2024-02-29 21:41:50,0.000447
66573,2024-02-29 21:55:23,0.003579
66955,2024-02-29 21:56:12,0.003042
149396,2024-02-29 22:00:57,0.000358
66562,2024-02-29 22:10:34,0.004474
593959,2024-02-29 22:54:32,0.000184


In [52]:
df['ltime']

0      2024-03-14 11:36:48
1732   2024-03-14 11:36:48
1733   2024-03-14 11:36:48
1734   2024-03-14 11:36:48
1735   2024-03-14 11:36:48
               ...        
8820   2024-03-14 12:55:34
8819   2024-03-14 12:55:34
8818   2024-03-14 12:55:34
8829   2024-03-14 12:55:34
9999   2024-03-14 12:55:34
Name: ltime, Length: 10000, dtype: datetime64[ns]

In [135]:
df = df[df['ltime'] - df['rtime'] < datetime.timedelta(minutes=1)]

In [136]:
len(df)

4315

In [13]:
(df['ltime'] - df['time'])[0]

Timedelta('-1 days +23:54:43')

In [19]:
x = df['time'][0]
x

Timestamp('2024-03-13 22:40:42')

In [20]:
y = df['ltime'][0]
y

Timestamp('2024-03-13 22:35:25')

In [27]:
(y - x) < datetime.timedelta(minutes=30)

True